# Experiment tracking

`SQLiteTracker` provides a powerful and flexible way to track ML experiments using a SQLite database. Allows you to use SQL as the query language, giving you a powerful tool for experiment comparison, and comes with plotting features to compare plots side-by-side and to combine plots for better comparison.

Read more about the motivations in our [blog post](https://ploomber.io/blog/experiment-tracking/), check out the [HN discussion.](https://news.ycombinator.com/item?id=33624018)

In [1]:
from sklearn_evaluation import SQLiteTracker

from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
tracker = SQLiteTracker('my_experiments.db')

In [3]:
X, y = datasets.make_classification(200, 10, n_informative=5, class_sep=0.65)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

models = [RandomForestClassifier(), LogisticRegression(), DecisionTreeClassifier()]

## Training and logging models

In [4]:
for m in models:
    model = type(m).__name__
    print(f'Fitting {model}')


    experiment = tracker.new_experiment()
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    # log a dictionary with log_many
    experiment.log_dict({'accuracy': acc, 'model': model, **m.get_params()})

Fitting RandomForestClassifier
Fitting LogisticRegression
Fitting DecisionTreeClassifier


Or use `.log(key, value)` to log individual values:

In [5]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc = accuracy_score(y_test, y_pred)

experiment = tracker.new_experiment()

# log individual values
experiment.log('accuracy', acc)
experiment.log('model', type(svc).__name__)

experiment.log_dict(svc.get_params())

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

## Displaying latest experiments

`tracker` shows last experiments:

In [6]:
tracker

uuid,created,parameters,comment
bed6a7c0,2022-11-24 17:07:11,,
9c8fd560,2022-11-24 17:07:11,"{""accuracy"": 0.6666666666666666, ""model"": ""LogisticRegression"", ""C"": 1.0, ""class_weight"": null, ""dual"": false, ""fit_intercept"": true, ""intercept_scaling"": 1, ""l1_ratio"": null, ""max_iter"": 100, ""multi_class"": ""auto"", ""n_jobs"": null, ""penalty"": ""l2"", ""random_state"": null, ""solver"": ""lbfgs"", ""tol"": 0.0001, ""verbose"": 0, ""warm_start"": false}",
290d60cf,2022-11-24 17:07:11,,
47418361,2022-11-24 17:07:11,"{""accuracy"": 0.6212121212121212, ""model"": ""DecisionTreeClassifier"", ""ccp_alpha"": 0.0, ""class_weight"": null, ""criterion"": ""gini"", ""max_depth"": null, ""max_features"": null, ""max_leaf_nodes"": null, ""min_impurity_decrease"": 0.0, ""min_samples_leaf"": 1, ""min_samples_split"": 2, ""min_weight_fraction_leaf"": 0.0, ""random_state"": null, ""splitter"": ""best""}",
cb578efc,2022-11-24 17:07:11,,


## Querying experiments with `.query()`

You can use SQL to query your experiments:

In [7]:
ordered = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
ORDER BY accuracy DESC
""")
ordered

,model,accuracy
uuid,,
0ff848e0,SVC,0.803030
089f404a,RandomForestClassifier,0.742424
c66dc0d3,RandomForestClassifier,0.742424
fbb4ce17,RandomForestClassifier,0.742424
5b78ce05,RandomForestClassifier,0.727273
64571332,SVC,0.727273
b7e0143a,LogisticRegression,0.712121
6881e884,DecisionTreeClassifier,0.681818
9c8fd560,LogisticRegression,0.666667


The query method returns a data frame with "uuid" as the index:

In [8]:
type(ordered)

pandas.core.frame.DataFrame

## Storing plots

You can log a confusion matrix and classification reports:

In [9]:
def fit(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    experiment = tracker.new_experiment()
    experiment.log_dict({'accuracy': acc, 'model': type(model).__name__, **model.get_params()})
    
    # log plots     
    experiment.log_confusion_matrix(y_test, y_pred)
    experiment.log_classification_report(y_test, y_pred)
    
    # log generic figure

fit(model=RandomForestClassifier(n_estimators=100))
fit(model=RandomForestClassifier(n_estimators=10))

In [10]:
tracker.recent(2)

,created,parameters,comment
uuid,,,
120d5fea,2022-11-24 17:07:15,None,None
b7e3b928,2022-11-24 17:07:15,"{""accuracy"": 0.696969696969697, ""model"": ""Rand...",None


## Rendering plots in table view

The `.query()` method also allows rendering plots in the table view:

In [11]:
results = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy,
       json_extract(parameters, '$.confusion_matrix') AS cm
FROM experiments
ORDER BY created DESC
LIMIT 2
""", as_frame=False, render_plots=True)

results

uuid,model,accuracy,cm
120d5fea,None,None,None
b7e3b928,RandomForestClassifier,0.696970,


## Side-by-side comparison

From the `.query()` results, you can extract a given column for a side by side comparison:

In [12]:
results.get("cm")

You can change the labels in the tabs with the `index_by` argument:

In [13]:
results.get("cm", index_by="accuracy")

## Combining plots

With a side-by-side comparison, it might be hard to spot the model performance differents, you can get individual experiments, extract their plots and combine them:

In [14]:
one = tracker.get("5b78ce05")
another = tracker.get("c66dc0d3")

In [16]:
one["confusion_matrix"] + another["confusion_matrix"]

In [17]:
one["classification_report"] + another["classification_report"]

Note that plot combination is only supported by plots generated via the `experiment.log_*` and not by the generic `experiment.log_figure` method.

## Adding comments

In [21]:
one.comment('This is some comment')

In [22]:
tracker.query("""
SELECT uuid,
       comment,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
ORDER BY accuracy, comment DESC
LIMIT 3
""")

,comment,model,accuracy
uuid,,,
4d882efc,This is some comment,None,None
bed6a7c0,None,None,None
290d60cf,None,None,None


## Pandas integration

### Getting recent experiments

The recent method also returns a data frame:

In [23]:
df = tracker.recent()
df

,created,parameters,comment
uuid,,,
4d882efc,2022-11-24 17:07:31,None,This is some comment
48e5f07e,2022-11-24 17:07:31,None,None
120d5fea,2022-11-24 17:07:15,None,None
b7e3b928,2022-11-24 17:07:15,"{""accuracy"": 0.696969696969697, ""model"": ""Rand...",None
8f395a89,2022-11-24 17:07:15,None,None


Pass `normalize=True` to convert the nested JSON dictionary into columns:

In [24]:
df = tracker.recent(normalize=True)
df

,created,accuracy,model,bootstrap,ccp_alpha,class_weight,criterion,max_depth,max_features,max_leaf_nodes,...,confusion_matrix.cm,confusion_matrix.normalize,confusion_matrix.target_names,confusion_matrix.version,classification_report.class,classification_report.matrix,classification_report.keys,classification_report.target_names,classification_report.version,comment
uuid,,,,,,,,,,,,,,,,,,,,,
4d882efc,2022-11-24 17:07:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This is some comment
48e5f07e,2022-11-24 17:07:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
120d5fea,2022-11-24 17:07:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
b7e3b928,2022-11-24 17:07:15,0.69697,RandomForestClassifier,True,0.0,NaN,gini,NaN,sqrt,NaN,...,"[[27, 11], [9, 19]]",False,"[Class 0, Class 1]",0.8.1dev,sklearn_evaluation.plot.ClassificationReport,"[[0.75, 0.7105263157894737, 0.7297297297297298...","[precision, recall, f1-score, support]","[0, 1]",0.8.1dev,None
8f395a89,2022-11-24 17:07:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


### Querying

You can also use the `.query()` method with `as_frame=True` (default value) to get a `pandas.DataFrame`

In [26]:
df = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
ORDER BY accuracy DESC
LIMIT 3
""")
df

,model,accuracy
uuid,,
0ff848e0,SVC,0.803030
e06016b7,RandomForestClassifier,0.772727
089f404a,RandomForestClassifier,0.742424


In [ ]:
# delete our example database
from pathlib import Path
Path('my_experiments.db').unlink()